In [ ]:
# default_exp tsp

# traveler salesman problem (tsp)

> This notebook is a toolset for tsp, now support three ways to solve tsp. 


In [ ]:
#hide
from nbdev.showdoc import *

## Function define part

In [ ]:
#export

import sys
import itertools
import random
import time
import matplotlib.pyplot as plt
import numpy as np
import math
from python_tsp.heuristics import solve_tsp_simulated_annealing

def dist(p1, p2):
    """
        calculate the distance between two waypoints.
        Args:
            p1 : point's (x,y) position.
            p2 : point's (x,y) position.
        Returns:
            distance between two points.
    """
    return math.sqrt(((p1-p2)**2).sum())

def distanceGenerate(point_set):
    """
        generate a distance matrix based on the waypoints.
        Args:
            point_set : a set that contains all waypoint.
        Returns:
            a square matrix, which shows the distance between pairs of waypoint.
    """
    return np.asarray([[dist(np.array(p1), np.array(p2)) for p2 in point_set] for p1 in point_set])

def sortWaypoint(permutation, point_set):
    """
        accoriding to permutation calculated by tsp solver, return new set of waypoint which is sorted.
        Args:
            permutation : the order of waypoints calculated by tsp solver.
            point_set : a set that contains all waypoint.
        Returns:
            new set of waypoint which is sorted.
    """
    return [x for _, x in sorted(zip(permutation, point_set))]

def solve_tsp_nearest_neighbor(distance_matrix):
    """
        calculate tsp problem based on nearest neighbor, an algorithm that solves tsp using greedy assumption.
        Args:
            distance_matrix : a square matrix, which shows the distance between pairs of waypoint.
        Returns:
            A tuple, (path, cost)
            cost : optimal cost of tsp
            path : a orderd list of waypoint index based on distance matrix.
    """
    path = [0]
    cost = 0
    N = distance_matrix.shape[0]
    mask = np.ones(N, dtype=bool) 
    mask[0] = False

    for i in range(N-1):
        last = path[-1]
        next_ind = np.argmin(distance_matrix[last][mask]) # find minimum of remaining locations
        next_loc = np.arange(N)[mask][next_ind] # convert to original location
        path.append(next_loc)
        mask[next_loc] = False
        cost += distance_matrix[last, next_loc]
        if(i == N-2):
            cost += distance_matrix[next_loc, 0]

    return path, cost

def solve_tsp_held_karp(distance_matrix):
    """
        calculate tsp problem based on Held-Karp, an algorithm that solves tsp using dynamic programming with memoization.
        Args:
            distance_matrix : a square matrix, which shows the distance between pairs of waypoint.
        Returns:
            A tuple, (path, cost)
            cost : optimal cost of tsp
            path : a orderd list of waypoint index based on distance matrix.
    """
    n = len(distance_matrix)
    C = {}

    # Set transition cost from initial state
    for k in range(1, n):
        C[(1 << k, k)] = (distance_matrix[0][k], 0)

    # Iterate subsets of increasing length and store intermediate results
    # in classic dynamic programming manner
    for subset_size in range(2, n):
        for subset in itertools.combinations(range(1, n), subset_size):
            bits = 0
            for bit in subset:
                bits |= 1 << bit
            # Find the lowest cost to get to this subset
            for k in subset:
                prev = bits & ~(1 << k)
                res = []
                for m in subset:
                    if m == 0 or m == k:
                        continue
                    res.append((C[(prev, m)][0] + distance_matrix[m][k], m))
                C[(bits, k)] = min(res)
    bits = (2**n - 1) - 1

    # Calculate optimal cost
    res = []
    for k in range(1, n):
        res.append((C[(bits, k)][0] + distance_matrix[k][0], k))
    opt, parent = min(res)

    # Backtrack to find full path
    path = []
    for i in range(n - 1):
        path.append(parent)
        new_bits = bits & ~(1 << parent)
        _, parent = C[(bits, parent)]
        bits = new_bits

    # Add implicit start state
    path.append(0)

    return list(reversed(path)), opt



## Testing example
we will move to 'data' directory to start all examples, this directory have been added to '.gitignoe' , so is convenient to test in local and light in cloud

In [ ]:
cd data

/home/arg/arg_robotics_tools/data


### Example task
we set a list that contains 11 waypoint(the first one is the starting point)

In [ ]:
point_set = [
    [0, 0],
    [-13, 10],
    [-7, 6],
    [-1, 20],
    [3, 15],
    [10, 18],
    [13, 14],
    [16, 10],
    [24, 14],
    [25, 17],
    [33, 8]
]

### Generate distance matrix
generate distance matrix for tsp_solver

In [ ]:
distance_matrix = distanceGenerate(point_set)
print(distance_matrix)

## Solve tsp
here display three ways to solve tsp
* Held Karp alogrithm(dynamic programming)
* simulated annealing alogrithm(heuristics alogrithm)
* nearest neighbor alogrithm(greedy alogrithm)

In [ ]:
permutation_hk, distance_hk = solve_tsp_held_karp(distance_matrix)
result_hk = sortWaypoint(permutation_hk, point_set)

In [ ]:
permutation_sa, distance_sa = solve_tsp_simulated_annealing(distance_matrix)
result_sa = sortWaypoint(permutation_sa, point_set)

In [ ]:
permutation_nn, distance_nn = solve_tsp_nearest_neighbor(distance_matrix)
result_nn = sortWaypoint(permutation_nn, point_set)

## Show result

In [ ]:
print(permutation_hk)
print(distance_hk)
print(result_hk)

[0, 7, 10, 9, 8, 6, 5, 4, 3, 1, 2]


In [ ]:
print(permutation_sa)
print(distance_sa)
print(result_sa)

In [ ]:
print(permutation_nn)
print(distance_nn)
print(result_nn)